<a href="https://colab.research.google.com/github/Alan-oliveir/previsao_cartao_fidelidade/blob/main/Projeto_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando os Dados

In [ ]:
# Importar bibliotecas
import sqlite3
import pandas as pd

# Conectar com banco de dados
conn = sqlite3.connect( 'database.db' )

In [ ]:
# Realizar consulta ao banco de dados
consulta_atividade = """

  SELECT
    *
  FROM
    flight_activity fa LEFT JOIN flight_loyalty_history flh ON (fa.loyalty_number = flh.loyalty_number)

 """
df_atividade = pd.read_sql_query( consulta_atividade, conn )

# Visualizar consulta
df_atividade.head()

In [ ]:
# Fechar conexão com banco de dados
conn.close()

## Inspecionando os Dados

In [ ]:
# Verificar o número de linhas do dataframe
df_atividade.shape[0]

In [ ]:
# Verificar o número de colunas do dataframe
df_atividade.shape[1]

In [ ]:
# Insights iniciais da estrutura do dataframe
df_atividade.info()

In [ ]:
df_atividade.loc[: , "distance"]

In [ ]:
# Soma e média da distancia
distancia_total = df_atividade.loc[: , "distance"].sum()
distancia_media = df_atividade.loc[: , "distance"].mean()

# Mínimo e máximo
distancia_min = df_atividade.loc[: , "distance"].min()
distancia_max = df_atividade.loc[: , "distance"].max()

print("Distância total: ", distancia_total, "km")
print("Distância média: ", distancia_media, "km")
print("Distância mínima: ", distancia_min, "km")
print("Distância máxima: ", distancia_max, "km")

## Preparação dos Dados

In [ ]:
type (df_atividade)

In [ ]:
# Visualizar número de campos nulos nas colunas
df_atividade.isna().sum()

In [ ]:
# Selecionar colunas numéricas
colunas = ["year",
           "month",
           "flights_booked",
           "flights_with_companions",
           "total_flights",
           "distance",
           "points_accumulated",
           "salary",
           "clv",
           "loyalty_card"]

df_colunas_numericas = df_atividade.loc[: , colunas]

In [ ]:
# Remover linhas com dados faltantes
df_dados_completos = df_colunas_numericas.dropna()

In [ ]:
# Verificar dados faltantes
df_dados_completos.isna().sum()

In [ ]:
df_dados_completos.shape[0]

## Machine Learning

In [ ]:
df_dados_completos.head()

In [ ]:
# Importar biblioteca tree
from sklearn import tree as tr

x_atributos = df_dados_completos.drop(columns="loyalty_card")
y_rotulo = df_dados_completos.loc[:, "loyalty_card"]

# Definição do algoritmo
modelo = tr.DecisionTreeClassifier(max_depth=5)

# Treinamento do algoritmo
modelo_treinado = modelo.fit(x_atributos, y_rotulo)

In [ ]:
tr.plot_tree(modelo_treinado, filled=True);

## Apresentação dos Resultados

In [ ]:
x_novo = x_atributos.sample()
pred = modelo_treinado.predict_proba(x_novo)

print("Probabilidades: Aurora = {:.1%}, Nova = {:.1%} e Star = {:.1%}.".format(pred[0][0], pred[0][1], pred[0][2]))

## Painel de Visualização

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import numpy as np

In [ ]:
def predict(*args):
  x_novo = np.array([args]).reshape(1, -1)
  previsao = modelo_treinado.predict_proba(x_novo)
  return{"Aurora":previsao[0][0], "Nova":previsao[0][1], "Star":previsao[0][2]}

with gr.Blocks() as demo:
  # Título do painel
  gr.Markdown(""" # Propensão de Compra """)

  with gr.Row():
    with gr.Column():
      gr.Markdown(""" # Atributos do cliente """)
      year =                    gr.Slider(label="year", minimum=2017, maximum=2018, step=1, randomize=True)
      month =                   gr.Slider(label="month", minimum=1, maximum=12, step=1, randomize=True)
      flights_booked =          gr.Slider(label="flights_booked", minimum=0, maximum=21, step=1, randomize=True)
      flights_with_companions = gr.Slider(label="flights_with_companions", minimum=0, maximum=11, step=1, randomize=True)
      total_flights =           gr.Slider(label="total_flights", minimum=0, maximum=32, step=1, randomize=True)
      distance =                gr.Slider(label="distance", minimum=0, maximum=6293, step=1, randomize=True)
      points_accumulated =      gr.Slider(label="points_accumulated", minimum=0.00, maximum=676.50, step=0.1, randomize=True)
      salary =                  gr.Slider(label="salary", minimum=58486.00, maximum=407228.00, step=0.1, randomize=True)
      clv =                     gr.Slider(label="clv", minimum=2119.89, maximum=83325.38, step=0.1, randomize=True)

      with gr.Row():

        with gr.Row():
          gr.Markdown(""" # Botão de Previsao """)
          predict_btn = gr.Button(value="Previsao")

    with gr.Column():
      gr.Markdown(""" # Propensão de Compra do Cliente """)
      label = gr.Label()

  # Botão predict
  predict_btn.click(
      fn = predict,
      inputs = [
          year,
          month,
          flights_booked,
          flights_with_companions,
          total_flights,
          distance,
          points_accumulated,
          salary,
          clv
          ],
      outputs = [label])

demo.launch()